In [36]:
%run config

#to see if scope is there
#dbutils.secrets.listScopes()
client_id = dbutils.secrets.get(scope = "key_vault_secrets", key = "clientid") 
client_secret = dbutils.secrets.get(scope = "key_vault_secrets", key = "clientsecret") 
new_refresh_token = dbutils.secrets.get(scope = "key_vault_secrets", key = "newrefreshtoken")
activity_id_path = dbutils.secrets.get(scope = "key_vault_secrets", key = "activityidpath") 
historical_activity_id_path = dbutils.secrets.get(scope = "key_vault_secrets", key = "historicalactivitydfpath") 
segment_effort_path = dbutils.secrets.get(scope = "key_vault_secrets", key = "segmenteffortpath") 

import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"


payload = {
    'client_id':  client_id,
    'client_secret': client_secret,
    'refresh_token': new_refresh_token,
    'grant_type': 'refresh_token',
    'f

### API Query to get more specific details for each activity, need to pass each activity off individually 

In [37]:
#get full activity dataset from what is written in storage, should be all activites
full_activity_dataset = spark.read.format("delta").load(historical_activity_id_path)

In [38]:
#grab all of the disinct activity IDs 
full_activity_ids = full_activity_dataset.select('activity_ids').distinct().rdd.flatMap(lambda x: x).collect()

In [43]:
subset = full_activity_ids[:25]

In [46]:
from pyspark.sql.functions import * 
import pandas as pd

In [47]:
#Move activity_id_list and segment_id_list out of the for loop
def query_segments(activity_ids):
    df = pd.DataFrame()
    activity_id_list =[]
    segment_id_list =[]
    for id in activity_ids:
        activity_id_urls = ("{}{}?include_all_efforts= True").format("https://www.strava.com/api/v3/activities/",id)
        header = {'Authorization': 'Bearer ' + access_token}
        param = {'per_page': 200, 'page': 1}
        my_activity = requests.get(activity_id_urls, headers=header, params=param).json()

        segment_effort_count =  len(my_activity['segment_efforts'])
        count = 0
        while count < segment_effort_count:

            activity_id = my_activity['segment_efforts'][count]['activity']['id']
            segment_id = my_activity['segment_efforts'][count]['id']
            activity_id_list.append(activity_id)
            segment_id_list.append(segment_id)
                  
            columns = ['segment_id', 'activity_id']
            extracted_data = [segment_id_list, activity_id_list]
            segment_df = pd.DataFrame.from_dict(dict(zip(columns, extracted_data)))

            df= df.append(segment_df)
            count += 1

    #convert pandas df to spark
        
    segment_spark_df = spark.createDataFrame(df)

    segment_spark_df = segment_spark_df.withColumn("ingest_file_name", lit("segment_efforts_ids")) \
                                .withColumn("ingested_at", lit(current_timestamp()))

    return segment_spark_df







In [48]:
hopefull = query_segments(subset)

In [70]:
hopefull.groupBy('activity_id').agg(countDistinct('segment_id')).show()

+-----------+-----------------+
|activity_id|count(segment_id)|
+-----------+-----------------+
| 7358811380|               34|
| 5448078140|                1|
| 4838912326|               12|
| 5408255638|                2|
| 7261304130|                2|
| 7411376407|                3|
| 7303733207|                1|
| 3865986737|              128|
| 3871052555|               14|
| 7399680604|                1|
| 4768097716|                2|
| 4414021787|                1|
| 7437771392|               71|
| 7480841356|              144|
| 7379581860|               22|
| 3876332481|              131|
| 4730507502|               10|
+-----------+-----------------+

In [66]:
activity_list = hopefull.select('activity_id').distinct().collect()
len(activity_list)
len(subset)

Out[31]: 25

In [ ]:
from distutils.log import error
segment_effort_path

try:
    historical_dataframe = spark.read.format("delta").load(segment_effort_path)
except:
    error

if (historical_dataframe.count() > 0):
    return historical_dataframe
else:
    


return dataset 

In [ ]:
import pandas as pd
try:
    #try to read from storage
    pd.read_csv('file.csv')
except:
    #print("No file") 
    try: 
        pd.read_csv('file.csv')
    except:
        print( "No File x2")
finally:
    print("1+1")


In [ ]:
import pandas as pd
try:
    #try to read from storage
    pd.read_csv('file.csv')
except: 
    try:
    #if that fails, write the current dataframe to storage
        write_dataframe_to_storage(historical_df_to_write,storagepath, "mergeSchema", "append" )
    except:
        print("No Dataframe")
    finally:
        historical_dataframe = spark.read.format("delta").load(storagepath)

In [ ]:
#run activites through api call

#

In [ ]:
#need to get distinct activity_ids and run them through the segment
#we make 1 api query in the first script, so gonna be allowed 99 with this run 

In [ ]:
#get all of the activity_ids, limit to 15 as thats how many we can run in a single query
#also need to query the activities we have written to segment storage, so as not to repeat 
#Strava API usage is limited on a per-application basis using both a 15-minute and daily request limit. The default rate limit allows 100 requests every 15 minutes, 
# with up to 1,000 requests per day.

#compare current activites vs what is written, 
#going to need to write some try and excepts for expecting these values in return 